# Exploring Hacker News Posts

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting. 

We'll compare these two types of posts to determine the following:
- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

In [1]:
from csv import reader

In [2]:
opened_data = open('hacker_news.csv')
read_data = reader(opened_data)
hn = list(read_data)

In [3]:
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


### Isolate header row

In [4]:
header = hn[0]
hn = hn[1:]

In [5]:
print(header)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [6]:
print(hn[:5])

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

### Identify posts for Ask HN and Show HN

In [7]:
ask_posts = []
show_posts = []
other_posts = []

In [8]:
for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


In [9]:
total_ask_comments = 0
for row in ask_posts:
    if row[4] != '':
        number_comments = int(row[4])
        total_ask_comments += number_comments
avg_ask_comments = total_ask_comments/len(ask_posts)
print(avg_ask_comments)

total_show_comments = 0
for row in show_posts:
    if row[4] != '':
        number_comments = int(row[4])
        total_show_comments += number_comments
avg_show_comments = total_show_comments/len(show_posts)
print(avg_show_comments)

14.038417431192661
10.31669535283993


On average, ask posts receive more comments than show posts.
Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

- Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.

### Step 1: Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.

In [10]:
import datetime as dt

In [11]:
result_list = []
for row in ask_posts:
    created = row[6]
    if row[4] == '':
        comments = 0
    else:
        comments = int(row[4])
    result_list.append((created,comments))

In [12]:
counts_by_hour = {}
comments_by_hour = {}

for pair in result_list:
    create_date = dt.datetime.strptime(pair[0],'%m/%d/%Y %H:%M')
    create_hour = dt.datetime.strftime(create_date,'%H')
    if create_hour in counts_by_hour:
        counts_by_hour[create_hour] += 1
        comments_by_hour[create_hour] += pair[1]
    else:
        counts_by_hour[create_hour] = 1
        comments_by_hour[create_hour] = pair[1]

### Step 2: Calculate the average number of comments ask posts receive by hour created.

In [13]:
avg_by_hour = []
for hour in counts_by_hour:
    avg = comments_by_hour[hour]/counts_by_hour[hour]
    avg_by_hour.append([hour,avg])
print(avg_by_hour)

[['10', 13.440677966101696], ['20', 21.525], ['02', 23.810344827586206], ['17', 11.46], ['01', 11.383333333333333], ['21', 16.009174311926607], ['22', 6.746478873239437], ['13', 14.741176470588234], ['07', 7.852941176470588], ['16', 16.796296296296298], ['12', 9.41095890410959], ['18', 13.20183486238532], ['11', 11.051724137931034], ['15', 38.5948275862069], ['23', 7.985294117647059], ['06', 9.022727272727273], ['09', 5.5777777777777775], ['19', 10.8], ['05', 10.08695652173913], ['00', 8.127272727272727], ['08', 10.25], ['03', 7.796296296296297], ['14', 13.233644859813085], ['04', 7.170212765957447]]


In [14]:
swap_avg_by_hour = []
for each in avg_by_hour:
    swap_avg_by_hour.append([each[1],each[0]])
print(swap_avg_by_hour)

[[13.440677966101696, '10'], [21.525, '20'], [23.810344827586206, '02'], [11.46, '17'], [11.383333333333333, '01'], [16.009174311926607, '21'], [6.746478873239437, '22'], [14.741176470588234, '13'], [7.852941176470588, '07'], [16.796296296296298, '16'], [9.41095890410959, '12'], [13.20183486238532, '18'], [11.051724137931034, '11'], [38.5948275862069, '15'], [7.985294117647059, '23'], [9.022727272727273, '06'], [5.5777777777777775, '09'], [10.8, '19'], [10.08695652173913, '05'], [8.127272727272727, '00'], [10.25, '08'], [7.796296296296297, '03'], [13.233644859813085, '14'], [7.170212765957447, '04']]


In [15]:
sorted_swap = sorted(swap_avg_by_hour, reverse = True)
print('Top 5 Hours for Ask Posts Comments')

for each in sorted_swap[:5]:
    date_hour = dt.datetime.strptime(each[1],'%H')
    date_hour = dt.datetime.strftime(date_hour,'%H:%M')
    template = '{hour}: {average:.2f} comments per cost'
    print(template.format(hour=date_hour,average=each[0]))

Top 5 Hours for Ask Posts Comments
15:00: 38.59 comments per cost
02:00: 23.81 comments per cost
20:00: 21.52 comments per cost
16:00: 16.80 comments per cost
21:00: 16.01 comments per cost


Creating an Ask Post at 15:00 is most likely to receive the most comments.

That's it for the guided steps! Here's a quick summary of what we accomplished in this guided project:

- We set a goal for the project.
- We collected and sorted the data.
- We reformatted and cleaned the data to prepare it for analysis.
- We analyzed the data.

Next steps:
- Determine if show or ask posts receive more points on average.
- Determine if posts created at a certain time are more likely to receive more points.
- Compare your results to the average number of comments and points other posts receive.
- Use Dataquest's data science project style guide to format your project.